In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import re

pd.set_option('display.max_columns', None)

In [2]:
# Set data path
data_path = '../data/interim/'

# Set constants
a = "Activities"
br = "Breating Rate"
hrv = "Heart Rate Variability"
skin = "Skin Temp"
gs = "General Sleep"
sl = "Sleep Levels"
gf = "Google Form"
mfp = "myFitnessPal"
stress = "Stress"


# Read Data

In [3]:
"""
DataManager Class

Attributes:
    - activities_df: DataFrame containing activity-related data from 'initial_clean_activities.csv'.
    - br_df: DataFrame containing breath rate data from 'initial_clean_br.csv'.
    - hrv_df: DataFrame containing heart rate variability data from 'initial_clean_hrv.csv'.
    - skin_df: DataFrame containing skin temperature data from 'initial_clean_tempSkin.csv'.
    - sleep_levels_df: DataFrame containing detailed sleep cycle data from 'initial_clean_sleep_levels.csv'.
    - sleep_gen_df: DataFrame containing general sleep data from 'initial_clean_sleep.csv'.
    - form_df: DataFrame containing user form data from 'initial_clean_gf.csv'.
    - mfp_df: DataFrame containing MyFitnessPal data from 'initial_clean_mfp.csv'.
    - stress_df: DataFrame containing stress score data from 'stress_score.csv'.
    - all_dfs: A dictionary mapping unique keys to the corresponding DataFrames for centralized access:
        - 'a': Activities DataFrame.
        - 'br': Breath rate DataFrame.
        - 'hrv': Heart rate variability DataFrame.
        - 'skin': Skin temperature DataFrame.
        - 'gs': General sleep DataFrame.
        - 'sl': Sleep levels DataFrame.
        - 'gf': Form DataFrame.
        - 'mfp': MyFitnessPal DataFrame.
        - 'stress': Stress score DataFrame.

Methods:
    - __init__(self, data_path):
        Initializes the `DataManager` instance by loading CSV files into corresponding DataFrames.

    - get_df(self, key):
        Retrieves a DataFrame from `all_dfs` based on the given key.
        Parameters:
            - key (str): The key representing the desired DataFrame.
        Returns:
            - DataFrame corresponding to the given key.
        Raises:
            - KeyError: If the key is not found in `all_dfs`.

    - update_df(self, key, new_df):
        Updates a specific DataFrame and ensures consistency in `all_dfs`.
        Parameters:
            - key (str): The key representing the DataFrame to update.
            - new_df (DataFrame): The updated DataFrame.
        Returns:
            - The updated DataFrame.
        Raises:
            - KeyError: If the key is not found in `all_dfs`.

    - __update_all_dfs(self):
        A private method to refresh the `all_dfs` dictionary after updates.

    - update_all_dfs(self, adfs):
        Updates all DataFrames in the `all_dfs` dictionary with a new set of DataFrames.
        Parameters:
            - adfs (dict): A dictionary containing the new DataFrames.
        Returns:
            - The updated `all_dfs` dictionary.
        Raises:
            - ValueError: If the number of DataFrames in `adfs` does not match `all_dfs`.
            - KeyError: If the keys in `adfs` do not match the keys in `all_dfs`.

    - get_dfs_attr_name(self, key):
        Retrieves the attribute name of a DataFrame based on the given key.
        Parameters:
            - key (str): The key representing the desired DataFrame.
        Returns:
            - The attribute name as a string, or None if the key is invalid.
"""

class DataManager:
    def __init__(self, data_path):
        self.activities_df = pd.read_csv(data_path + 'initial_clean_activities.csv')
        self.br_df = pd.read_csv(data_path + 'initial_clean_br.csv')
        self.hrv_df = pd.read_csv(data_path + 'initial_clean_hrv.csv')
        self.skin_df = pd.read_csv(data_path + 'initial_clean_tempSkin.csv')
        self.sleep_levels_df = pd.read_csv(data_path + 'intial_clean_sleep_levels.csv')
        self.sleep_gen_df = pd.read_csv(data_path + 'initial_clean_sleep.csv')
        self.form_df = pd.read_csv(data_path + 'initial_clean_gf.csv')
        self.mfp_df = pd.read_csv(data_path + 'initial_clean_mfp.csv')
        self.stress_df = pd.read_csv(data_path + 'stress_score.csv')
        
        self.all_dfs = {
            a : self.activities_df,
            br : self.br_df,
            hrv: self.hrv_df,
            skin: self.skin_df,
            gs: self.sleep_gen_df,
            sl : self.sleep_levels_df,
            gf: self.form_df,
            mfp: self.mfp_df,
            stress : self.stress_df,
        }
    
    def get_df(self, key):
        #Check if key returns a non None value
        if self.get_dfs_attr_name(key):

            #return all_dfs[key] since we know that key is valid
            return self.all_dfs[key]
        else:
            raise KeyError(f"Key '{key}' not found in DataManager.")
    
    def update_df(self, key, new_df):
        if self.get_dfs_attr_name(key):
            
            attr_name = self.get_dfs_attr_name(key)
            if hasattr(self, attr_name):
                setattr(self, attr_name, new_df)
                self.__update_all_dfs()
        else:
            raise KeyError(f"Key '{key}' not found in DataManager.")
        
        return new_df
    
    def __update_all_dfs(self):
        self.all_dfs = {
            a : self.activities_df,
            br : self.br_df,
            hrv: self.hrv_df,
            skin: self.skin_df,
            gs: self.sleep_gen_df,
            sl : self.sleep_levels_df,
            gf: self.form_df,
            mfp: self.mfp_df,
            stress : self.stress_df,
        }

    def update_all_dfs(self, adfs):
        if len(self.all_dfs) != len(adfs):
            raise ValueError("Wrong amounts of dfs")
        
        if self.all_dfs.keys() != adfs.keys():
            raise KeyError("Wrong set of keys to map")
        
        for key, value in adfs.items():
            self.update_df(key, value)

        return self.all_dfs
        

    def get_dfs_attr_name(self, key):
        dfs_list = {
            a : "activities_df",
            br : "br_df",
            hrv: "hrv_df",
            skin: "skin_df",
            gs: "sleep_gen_df",
            sl : "sleep_levels_df",
            gf: "form_df",
            mfp: "mfp_df",
            stress : "stress_df",
        }

        return dfs_list.get(key, None)

    

data = DataManager(data_path)

activities_df = data.get_df(a)
br_df = data.get_df(br)
hrv_df = data.get_df(hrv)
skin_df = data.get_df(skin)
sleep_gen_df = data.get_df(gs)
sleep_levels_df = data.get_df(sl)
form_df = data.get_df(gf)
mfp_df = data.get_df(mfp)
stress_df = data.get_df(stress)






# Basic Preprocessing

In [4]:
# Limit general sleep dataframe to main sleeping sessions (rid of naps)
sleep_gen_df = sleep_gen_df[sleep_gen_df['isMainSleep']]
sleep_gen_df = data.update_df(gs, sleep_gen_df)

# Using sleep gen df as a reference, keep sleep level records of only main sleeping sessions 
sleep_levels_df = sleep_levels_df[sleep_levels_df["sleep_id"].isin(set(sleep_gen_df["sleep_id"]))]
sleep_levels_df = data.update_df(sl, sleep_levels_df)
# Drop columns since only one instance was recorded in them
form_df.drop(columns=['stress_responsiveness', 'stress_exertion', 'stress_sleep', 'stress_mgmt_score'], inplace=True)

# Get rid of data points where calculations of stress scores was lost, in this case the missing values provide no value
stress_df = stress_df[stress_df['CALCULATION_FAILED'] == False]
stress_df = data.update_df(stress, stress_df)

In [5]:
for key, df in data.all_dfs.items():
    
    for col in df.columns:
        if re.search("date", col, re.IGNORECASE):
            print(f"In {key}, date column(s) is: {col}")
            print(f"Example of column format: {df[col].iloc[0]}")
            print()

print("In Activities data, date column is: startTime")
print(f"Example of column format: {(activities_df['startTime'].iloc[0])}")


In Breating Rate, date column(s) is: dateTime
Example of column format: 2023-11-23

In Heart Rate Variability, date column(s) is: dateTime
Example of column format: 2023-11-23

In Skin Temp, date column(s) is: dateTime
Example of column format: 2023-11-26

In General Sleep, date column(s) is: dateOfSleep
Example of column format: 2023-11-23

In General Sleep, date column(s) is: datetime_key
Example of column format: 2023-11-23T12:51:00

In Sleep Levels, date column(s) is: dateTime
Example of column format: 2023-11-23T02:29:30.000

In Google Form, date column(s) is: date
Example of column format: 7/23/2024

In myFitnessPal, date column(s) is: date
Example of column format: 2024-04-15

In Stress, date column(s) is: DATE
Example of column format: 2023-11-23T00:00:00

In Stress, date column(s) is: UPDATED_AT
Example of column format: 2023-11-23T19:51:56.998

In Activities data, date column is: startTime
Example of column format: 2023-11-25T18:59:25.000-05:00


In [6]:
list_of_date_cols = ["date", "DATE", "dateTime", "startTime"]

all_dfs = data.all_dfs

for key, df in all_dfs.items():
    if key == a:
        all_dfs[key]["normalized_datetime"] = pd.to_datetime(all_dfs[key]["startTime"], errors="coerce", utc=True)
        all_dfs[key]["normalized_datetime"] = all_dfs[key]["normalized_datetime"].dt.tz_convert("America/New_York")
        continue
    for col in df.columns:
        if col in list_of_date_cols:
            print(key)
            all_dfs[key]["normalized_datetime"] = pd.to_datetime(all_dfs[key][col], errors="coerce")
        
#print(all_dfs)
all_dfs = data.update_all_dfs(all_dfs)
    #print(all_dfs.head())

Breating Rate
Heart Rate Variability
Skin Temp
General Sleep
Sleep Levels
Google Form
myFitnessPal
Stress
{'Activities':      activityTypeId activityName  calories   steps  averageHeartRate  \
0             90013         Walk       562  4874.0             129.0   
1             90013         Walk       125  1142.0             104.0   
2             90009          Run        65   536.0             110.0   
3             90013         Walk       186  1615.0             124.0   
4             90013         Walk       135  1405.0             107.0   
..              ...          ...       ...     ...               ...   
437           90013         Walk       121  1103.0             101.0   
438           90013         Walk       142  1468.0             123.0   
439           90013         Walk       137  1618.0             110.0   
440           90013         Walk        99  1167.0             104.0   
441           90013         Walk       394  4109.0             113.0   

     duration 

In [14]:
# Next step is to replace google form answers with numbers for easier analysis

form_df.head()

,timestamp,date,last_sleep_feel,last_sleep_score,daily_particular_qualities,day_feel,ef_status,ef_description,dissatisfied_status,normalized_datetime
0,7/23/2024 23:22:15,7/23/2024,Bad,NaN,"Mentally Tired, Physically Tired",Good,No,NaN,"None / little work done., Tired.",2024-07-23
1,7/24/2024 23:06:55,7/24/2024,Very Good,NaN,Felt Great Today,Very Good,Yes,Dad made me run errands all day.,None / little work done.,2024-07-24
2,7/26/2024 9:47:16,7/25/2024,Mid,NaN,Normal,Good,Yes,Dad,None / little work done.,2024-07-25
3,7/26/2024 22:51:08,7/26/2024,Very Good,NaN,Felt Great Today,Very Good,No,NaN,Not dissatisfied with anything.,2024-07-26
4,7/29/2024 0:31:44,7/27/2024,Good,NaN,Normal,Good,No,A lot of work to do.,None / little physical activity done.,2024-07-27


In [16]:
form_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   timestamp                   51 non-null     object        
 1   date                        51 non-null     object        
 2   last_sleep_feel             44 non-null     object        
 3   last_sleep_score            37 non-null     float64       
 4   daily_particular_qualities  44 non-null     object        
 5   day_feel                    44 non-null     object        
 6   ef_status                   44 non-null     object        
 7   ef_description              14 non-null     object        
 8   dissatisfied_status         43 non-null     object        
 9   normalized_datetime         51 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 4.1+ KB


In [20]:
for col in form_df.columns[2:]:
    if form_df[col].dtype == 'object':
        print(col)
        print(form_df[col].value_counts())
        print()

last_sleep_feel
last_sleep_feel
Mid          16
Bad          12
Good          8
Very Good     5
Very Bad      3
Name: count, dtype: int64

daily_particular_qualities
daily_particular_qualities
Mentally Tired, Physically Tired                               11
Mentally Tired, Physically Tired, Low Motivation                8
Normal                                                          6
Felt Great Today                                                3
Normal, Low Motivation                                          3
Normal, Felt Great Today                                        3
Mentally Tired, Physically Tired, Felt Sick                     2
Mentally Tired, Physically Tired, Low Motivation, Felt Sick     2
Mentally Tired                                                  1
Low Motivation                                                  1
Mentally Tired, Normal                                          1
Normal, Low Motivation, Felt Sick                               1
Normal, Felt Si

In [ ]:
last_sleep_feel_mapping = {
    'Very Bad': -2,
    'Bad': -1,
    'Mid': 0,
    'Good': 1,
    'Very Good': 2
}

day_feel_mapping = {
    'Very Good': 2,
    'Good': 1,
    'Mid': 0,
    'Bad': -1,
    'Very Bad' : -2
}

daily_particular_qualities_mapping = {
    'Mentally Tired': -1,
    'Physically Tired': -1,
    'Low Motivation': -1,
    'Normal': 0,
    'Felt Great Today': 2,
    'Felt Sick': -3,
    'Stressed': -2
}

dissatisfied_sentiment_mapping = {
    'None / little work done.': -1,
    'None / little physical activity done.': -1,
    'Tired.': -1,
    'Not dissatisfied with anything.': 1,
    'Did not achieve daily goal.' : -1,
    'Did not have fun / do hobbies.' : -1,
}



